In [1]:
import re
import math
from collections import Counter

In [2]:
filename = "sample.txt"
# filename = "input.txt"
with open(filename, encoding="utf-8") as f:
    data = f.read()

# lines = data.strip().split("\n")

https://adventofcode.com/2024/day/14

In [3]:
robot_pattern = r"p=(-?\d+),(-?\d+) v=(-?\d+),(-?\d+)"
initial_robots = []
for m in re.findall(robot_pattern, data):
    px, py, vx, vy = map(int, m)
    pos = complex(px, py)
    step = complex(vx, vy)
    initial_robots.append((pos, step))

In [4]:
## Part 1
# Since robots don't collide and the area wraps around, we should be able to calculate the final position in one step per robot
# Final pos = (start pos + n * step) mod x,y
# Result after 100 seconds = mul(number of robots in each quadrant)

def teleport(pos: complex, x_hi: int, y_hi: int) -> complex:
    return complex(pos.real % x_hi, pos.imag % y_hi)

def take_steps(start_pos: complex, step: complex, n: int) -> complex:
    return start_pos + (step * n)

def safety_factor(robots: list[complex], x_hi, y_hi):
    x_mid = (x_hi // 2)
    y_mid = (y_hi // 2)
    # 4 quadrants NW, NE, SW, SE
    # (<x, <y), (>x, <y), ...
    quadrants = {(x, y): 0 for x in (False, True) for y in (False, True)}
    for pos in robots:
        x, y = pos.real, pos.imag
        # Skip any robots exactly on the middle-lines
        if (x == x_mid) or (y == y_mid):
            continue
        quadrants[(x < x_mid), (y < y_mid)] += 1
    
    result = math.prod(quadrants.values())
    return result

In [ ]:
# Grid bounds (note still 0-indexed, the bound here is exclusive 0 <= x < x_hi)
x_hi, y_hi = 11, 7  # Sample
# x_hi, y_hi = 101, 103
n_steps = 100
robots = [teleport(take_steps(robot, step, n_steps), x_hi, y_hi) for robot, step in initial_robots]
safety_factor(robots, x_hi, y_hi)

In [6]:
## Part 2
# The robots should arrange themselves into a picture of an Xmas tree
# What's the fewest number of seconds for this to happen??
def visualise(robots: list[complex], x_hi: int, y_hi: int) -> str:
    result = []
    grid = Counter(robots)
    for y in range(y_hi):
        for x in range(x_hi):
            n_robots = grid.get(complex(x, y), '.')
            result.append(str(n_robots))
            # print(n_robots, end="")
        result.append("\n")
        # print()
    return "".join(result)

In [7]:
safety_factors = []
results = []
for n_steps in range(0, 10_000):
    robots = [teleport(take_steps(robot, step, n_steps), x_hi, y_hi) for robot, step in initial_robots]
    # Arbitrary heuristic (taken from reddit): Manually check only results with an above-average safety factor (more robots in the centre)
    safety = safety_factor(robots, x_hi, y_hi)
    results.append((n_steps, robots, safety))
    safety_factors.append(safety)

sorted_results = sorted([r for r in results], key=lambda r: r[2])

In [ ]:
# Thankfully for my input, it was the step with the highest safety!
for i in range(100):
    n_steps, robots, safety = sorted_results[i]
    print(f"---- Step {n_steps}, Safety {safety} ----")
    print(visualise(robots, x_hi, y_hi))
    print()